# Install Repository

In [ ]:
#Run this block to check GPU you've been assigned
!nvidia-smi

In [ ]:
#first time run
!git clone https://github.com/antonioverdi/MLReproChallenge.git
#At the moment all requirements are met by Google Colab already I believe
import os
os.chdir("MLReproChallenge")
!pip install -r requirements.txt

In [ ]:
#subsequent runs
!git pull
!pip install -r requirements.txt

# Train Models
### main.py Arguments
<table>
    <tr><th>Argument</th><th>Default</th><th>help</th></tr>
    <tr><td>--arch, - a</td><td>str: 'resnet56'</td><td>model architecture</td></tr>
    <tr><td>-j, --workers</td><td>int: 4</td><td>number of data loading workers</td></tr>
    <tr><td>--epochs</td><td>int: 182</td><td>number of total epochs to run</td></tr>
    <tr><td>--start-epoch</td><td>int: 0</td><td>manual epoch number, for restarts</td></tr>
    <tr><td>-b, --batch-size</td><td>int: 128</td><td>mini-batch size</td></tr>
    <tr><td>--lr, --learning-rate</td><td>float: 0.1</td><td>initial learning rate</td></tr>
    <tr><td>--momentum</td><td>float: 0.9</td><td>momentum</td></tr>
    <tr><td>--weight-decay, --wd</td><td>float: 2e-4</td><td>weight decay</td></tr>
    <tr><td>--print-freq, -p</td><td>int: 50</td><td>print frequency</td></tr>
    <tr><td>--resume</td><td>str: ' '</td><td>path to latest checkpoint</td></tr>
    <tr><td>-e, --evaluate</td><td>bool: False</td><td>evaluate model on validation set</td></tr>
    <tr><td>--pretrained</td><td>bool: False</td><td>use pre-trained model</td></tr>
    <tr><td>--half</td><td>bool: False</td><td>use half-precision float(16-bit)</td></tr>
    <tr><td>--save-dir</td><td>str: 'save_temp'</td><td>Directory used to save trained models</td></tr>
    <tr><td>--save-every</td><td>int: 10</td><td>Save checkpoint at every specified number of epochs</td></tr>
    <tr><td>--colab</td><td>bool: False</td><td>Set this to true when running in Google Colab</td></tr>
    <tr><td>--snip</td><td>bool: False</td><td>Set this to true to run SNIP experiments</td></tr>
    <tr><td>--snip_compression</td><td>float: 0.5</td><td>eg. compression of 0.25 retains 25 percent of weights</td></tr>
   </table>

In [ ]:
%%time
!python main.py --colab

# Test Models
### test.py Arguments
(currently no arguments. test.py needs to be made more colab friendly)
<table>
    <tr><th>Argument</th><th>Default</th><th>help</th></tr>
    <tr><td>--arch, - a</td><td>str: 'resnet56'</td><td>model architecture</td></tr>
   </table>

<h4>SNIP experiments</h4>

In [ ]:
!python main.py --colab --snip --arch="resnet56_snip" --save-dir="SNIP_checkpoints" --save-every=25

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import types
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
X = torch.rand((10,5))
y = torch.ones(10, dtype=torch.long)
y[:5] = 0

In [3]:
class TestModel(nn.Module):
    def __init__(self, init_strategy="kaiming"):
        super(TestModel, self).__init__()
        self.layer1 = nn.Linear(5,3)
        self.layer2 = nn.Linear(3,5)
        if init_strategy == "kaiming":
            self.apply(_kaiming_weights_init)
        elif init_strategy == "xavier":
            self.apply(_xavier_weights_init)
            
    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = self.layer2(out)
        return out

In [36]:
model = TestModel(init_strategy="xavier")
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
masks = snip_mask(model, X, y, 0.50)
apply_snip(model, masks)


In [15]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
									 std=[0.229, 0.224, 0.225])
train_loader = torch.utils.data.DataLoader(
		datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
			transforms.RandomHorizontalFlip(),
			transforms.RandomCrop(32, 4),
			transforms.ToTensor(),
			normalize,
		]), download=True),
		batch_size=16, shuffle=True,
		num_workers=1, pin_memory=False)

Files already downloaded and verified


In [37]:
print(next(model.parameters()))

Parameter containing:
tensor([[-0.6454, -0.4165,  0.0000,  1.1258,  0.1893],
        [-0.8022,  0.0000,  0.6923,  0.0000,  0.0000],
        [ 0.1831, -0.2365, -0.6341,  0.8430,  0.0000]], requires_grad=True)
